In [18]:
from datetime import datetime
from bs4 import BeautifulSoup
from io import BytesIO
import pymysql
import requests
import json
import pandas as pd
import time
import datetime
import re
import csv
import os

In [51]:
from datetime import datetime

tt = datetime.today()
today = tt.strftime("%Y-%m-%d")

today

'2023-11-14'

# 인터파크 랭킹

In [124]:
interpark_url = "https://tickets.interpark.com/contents/ranking?genre=musical"
r_i = requests.get(url=interpark_url)
tmp_list = [] # 합칠 리스트 생성

if r_i.status_code ==200:
    print("Connect Success!")
    html_i = r_i.content

soup_i = BeautifulSoup(html_i, 'html.parser')

# TOP3는 가장 위에 가로로 배열 -> tag_v
# TOP4~10은 그 아래라 세로로 배열 -> tag_h
tag_v = soup_i.findAll('div', class_="ranking-vertical-item_rankingItem__llUL_")
tag_h = soup_i.findAll('ul', class_='ranking-horizontal-item_rankingContents__z3wFG')

try:
    # 가로 TOP 3 리스트
    for i in range(len(tag_v)):
        name_v = tag_v[i].find('li', class_='ranking-vertical-item_rankingGoodsName__m0gOz').text
        date_v =tag_v[i].find('div', class_='ranking-vertical-item_dateWrap__uZGMU').text
        location_v = tag_v[i].find('li', class_='ranking-vertical-item_placeName__4sHRS').text
        img_v = tag_v[i].find('div', class_='ranking-vertical-item_imageWrap__R6lkF').find('img')["src"]
        id_v = img_v.split('%')[8].split('_')[0].split('F')[1]
        goods_v = f"https://tickets.interpark.com/goods/{id_v}"
        img_url_v = f"https://tickets.interpark.com{img_v}"
        
        v_dict = {
            'title': name_v,
            'date' : date_v,
            'location' : location_v,
            'poster_url' : img_url_v,
            'goods_url' : goods_v
        }
        tmp_list.append(v_dict)
    
    # 세로 TOP 4~10 리스트
    for j in range(7):
        name_h = tag_h[j].find('li', class_='ranking-horizontal-item_rankingTicketTitle__omJYh').text
        date_h = date_h = tag_h[j].find('div', class_='ranking-horizontal-item_dateWrap__tRsWh').text
        location_h = tag_h[j].find('li', class_='ranking-horizontal-item_placeName__zb9kN').text
        img_h = tag_h[j].find('div', class_='ranking-horizontal-item_imageWrap__owTl6').find('img')['src']
        id_h = img_h.split('%')[8].split('_')[0].split('F')[1]
        goods_h = f"https://tickets.interpark.com/goods/{id_h}"
        img_url_h = f"https://tickets.interpark.com{img_h}"
        
        h_dict = {
            'title': name_h,
            'date' : date_h,
            'location' : location_h,
            'poster_url' : img_url_h,
            'goods_url' : goods_h
        }
        tmp_list.append(h_dict)

except Exception as  e:
    print(f"에러 발생 : {e}")

interpark_df = pd.DataFrame(tmp_list)

# rank, site_name 컬럼 추가 및 컬럼 순서 변경
interpark_df['rank'] = range(1, len(interpark_df) + 1)
interpark_df['site_name'] = '인터파크'
interpark_df = interpark_df[['site_name', 'rank', 'title', 'date', 'location', 'poster_url', 'goods_url']]

# 데이터 업데이트 날짜 기입
interpark_df['update_date']=today
interpark_df

Connect Success!


,site_name,rank,title,date,location,poster_url,goods_url,update_date
0,인터파크,1,뮤지컬 드라큘라 (Dracula：The Musical),2023.12.6~2024.3.3,샤롯데씨어터,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23013512,2023-11-14
1,인터파크,2,태양의서커스 〈루치아〉,2023.10.25~12.31,잠실종합운동장 내 빅탑,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23008707,2023-11-14
2,인터파크,3,뮤지컬 〈렌트〉,2023.11.11~2024.2.25,coex 신한카드 artium,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23013171,2023-11-14
3,인터파크,4,뮤지컬 〈몬테크리스토〉,2023.11.21~ 2024.2.25,충무아트센터 대극장,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23014423,2023-11-14
4,인터파크,5,뮤지컬 〈레미제라블〉,2023.11.30~ 2024.3.10,블루스퀘어 신한카드홀,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23012526,2023-11-14
5,인터파크,6,뮤지컬 〈빨래〉,2023.10.12~ 2024.5.26,대학로 유니플렉스 2관,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23012969,2023-11-14
6,인터파크,7,뮤지컬 〈레베카〉 10주년 기념공연,2023.8.19~ 11.19,블루스퀘어 신한카드홀,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23008837,2023-11-14
7,인터파크,8,뮤지컬 〈타오르는 어둠 속에서〉,2023.8.26~ 11.26,링크아트센터 페이코홀,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23009934,2023-11-14
8,인터파크,9,태양의서커스 〈루치아〉 - 부산,2024.1.13~ 2.4,신세계 센텀시티 내 빅탑,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23012373,2023-11-14
9,인터파크,10,뮤지컬 〈컴프롬어웨이〉,2023.11.28~ 2024.2.18,광림아트센터 BBCH홀,https://tickets.interpark.com/contents/_next/i...,https://tickets.interpark.com/goods/23013839,2023-11-14


# 티켓 링크 랭킹

In [135]:
url_t = "https://mapi.ticketlink.co.kr/mapi/ranking/genre/daily?categoryId=10&categoryId2=16&categoryId3=0&menu=RANKING"

r_t = requests.get(url=url_t)

if r_t.status_code == 200:
    print("Connect Success!")
    
    # json 데이터 df 변경
    data = json.loads(r_t.text)['data']['rankingList']
    ticketlink_df = pd.DataFrame(data)

# 필요한 컬럼 추출 및 전처리
columns = ['productId', 'productName', 'startDate', 'endDate', 'hallName', 'imgUrl']
ticketlink_df = ticketlink_df[columns]

ticketlink_df['imgUrl'] = 'https:' + ticketlink_df['imgUrl']
ticketlink_df['productId'] = 'https://www.ticketlink.co.kr/product/' + ticketlink_df['productId'].astype(str)
ticketlink_df['startDate'] = ticketlink_df['startDate'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y.%m.%d.'))
ticketlink_df['endDate'] = ticketlink_df['endDate'].apply(lambda x: datetime.fromtimestamp(x / 1000).strftime('%Y.%m.%d.'))
ticketlink_df['date'] = ticketlink_df['startDate'] + ' ~ ' + ticketlink_df['endDate']
ticketlink_df.drop(columns=['startDate'], inplace=True)
ticketlink_df.drop(columns=['endDate'], inplace=True)

# 'rank', 'siteName' 컬럼 추가
ticketlink_df['rank'] = range(1, len(ticketlink_df) + 1)
ticketlink_df['site_name'] = '티켓링크'

# 컬럼 순서 변경
columns = ['site_name', 'rank', 'productName', 'date', 'hallName', 'imgUrl', 'productId']
ticketlink_df = ticketlink_df[columns]

# 컬럼명 변경
ticketlink_df.rename(columns={'productName': 'title', 'hallName': 'location', 'productId': 'goods_url', 'imgUrl':'poster_url'}, inplace=True)

ticketlink_df['update_date'] = today
ticketlink_df

Connect Success!


,site_name,rank,title,date,location,poster_url,goods_url,update_date
0,티켓링크,1,뮤지컬 드라큘라 (Dracula：The Musical),2023.12.05. ~ 2024.03.02.,샤롯데씨어터,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46101,2023-11-14
1,티켓링크,2,뮤지컬 <문스토리>,2023.09.25. ~ 2023.12.09.,드림아트센터 4관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/45428,2023-11-14
2,티켓링크,3,뮤지컬 <타오르는 어둠 속에서>,2023.08.25. ~ 2023.11.25.,링크아트센터 페이코홀,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/44995,2023-11-14
3,티켓링크,4,뮤지컬 <시스터 액트>,2023.11.20. ~ 2024.02.10.,디큐브 링크아트센터,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46006,2023-11-14
4,티켓링크,5,뮤지컬 <스모크>,2023.11.14. ~ 2024.02.03.,링크아트센터 벅스홀,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46233,2023-11-14
5,티켓링크,6,뮤지컬 <그날들> 10주년 기념 공연 - 고양,2023.12.21. ~ 2023.12.23.,고양아람누리 아람극장,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46949,2023-11-14
6,티켓링크,7,광주시립소년소녀합창단 제140회 정기공연 창작뮤지컬 <나를 노래해>,2023.11.30. ~ 2023.12.01.,빛고을 시민문화관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46914,2023-11-14
7,티켓링크,8,뮤지컬<후크>,2023.09.04. ~ 2023.11.25.,대학로 아트원씨어터 2관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/45221,2023-11-14
8,티켓링크,9,뮤지컬 <결투>,2023.12.01. ~ 2024.02.24.,예스24스테이지 2관,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46844,2023-11-14
9,티켓링크,10,뮤지컬 <스토리오브마이라이프>,2023.11.29. ~ 2024.02.17.,두산아트센터 연강홀,https://image.toast.com/aaaaab/ticketlink/TKL_...,https://www.ticketlink.co.kr/product/46043,2023-11-14


# 예스24티켓 랭킹

In [136]:
# yes_url = "http://ticket.yes24.com/New/Rank/Ranking.aspx?genre=15457"

yes_url = "http://ticket.yes24.com/New/Rank/Ajax/RankList.aspx"

payload = {
    "pt": "1"
    ,"ci": "16"
    ,"et": "2023-11-{}".format(today[-2:])
}

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

r_y = requests.post(yes_url, payload ,headers= header)

if r_y.status_code == 200:
    print("Connect Success!")
    soup1 = BeautifulSoup(r_y.text, "html.parser")
    # 랭킹 1~3위 데이터
    best123 = soup1.find("div", class_="rank-best")
    # 랭킹 4~10위 데이터
    list410 = soup1.find("div", class_="rank-list")

data_list = []

# try:
# 랭킹 1~3위 추출
for i in range(3):
    y_dict = {
        'site_name' : '예스24티켓',
        'rank' : None,
        'title': None,
        'date' : None,
        'location' : None,
        'poster_url' : None,
        'goods_url' : None,
        'update_date' : today
    }
    
    title_y = best123.find_all("p", class_="rlb-tit")[i].get_text()
    date_y = best123.find_all('p', class_="rlb-sub-tit")[i].get_text(separator='|', strip=True).split('|')[0]
    location_y = best123.find_all('p', class_="rlb-sub-tit")[i].get_text(separator='|', strip=True).split('|')[1]
    img_url_y = best123.find_all("span", class_="rank-best-img")[i].find("img",class_="rank-best-img").get("src")
    goods_y = best123.find("a", title = title_y).get("href").split("=")[1]
    rank_y = best123.find_all("p", class_ = "rank-best-number")[i].find("span").get_text()[:-1]

    y_dict = {
        'site_name' : '예스24티켓',
        'rank' : rank_y,
        'title': title_y,
        'date' : date_y,
        'location' : location_y,
        'poster_url' : img_url_y,
        'goods_url' : "http://ticket.yes24.com/Perf/{}".format(goods_y),
        'update_date' : today
    }

    data_list.append(y_dict)


for i in range(0,31,5):
    y_dict = {
        'site_name' : '예스24티켓',
        'rank' : None,
        'title': None,
        'date' : None,
        'location' : None,
        'poster_url' : None,
        'goods_url' : None,
        'update_date' : today
    }
    
    title_y = list410.find_all("div",class_=False)[i].find("p", class_="rank-list-tit").get_text()
    date_y = list410.find_all("div",class_=False)[i].find_all("p")[-1].get_text(separator='|', strip=True).split('|')[0]
    location_y = list410.find_all("div",class_=False)[i].find_all("p")[-1].get_text(separator='|', strip=True).split('|')[1]
    img_url_y = list410.find("img", alt=title_y).get("src")
    goods_y = list410.find_all("div",class_=False)[i].find("p", class_="rank-list-tit").find("a").get("href").split("=")[1]
    rank_y = list410.find_all("div",class_=False)[i].find_all("p")[0].get_text()[:-1]
    
    y_dict = {
        'site_name' : '예스24티켓',
        'rank' : rank_y,
        'title': title_y,
        'date' : date_y,
        'location' : location_y,
        'poster_url' : img_url_y,
        'goods_url' : "http://ticket.yes24.com/Perf/{}".format(goods_y),
        'update_date' : today
    }

    data_list.append(y_dict)

# except Exception as e:
#     print(e)

yes24_df= pd.DataFrame(data_list)

print("끝")
yes24_df

Connect Success!
끝


,site_name,rank,title,date,location,poster_url,goods_url,update_date
0,예스24티켓,1,뮤지컬 〈드라이 플라워〉,2023.11.07~2024.01.07,대학로 드림아트센터 3관,http://tkfile.yes24.com/upload2/perfblog/20231...,http://ticket.yes24.com/Perf/47290,2023-11-14
1,예스24티켓,2,태양의서커스 〈루치아〉,2023.10.25~2023.12.31,잠실종합운동장 내 빅탑,http://tkfile.yes24.com/upload2/perfblog/20230...,http://ticket.yes24.com/Perf/46305,2023-11-14
2,예스24티켓,3,오페레타 뮤지컬 〈판타지아〉 겨울시즌,2023.11.11~2023.12.31,국립중앙박물관 내 극장용,http://tkfile.yes24.com/upload2/perfblog/20231...,http://ticket.yes24.com/Perf/47090,2023-11-14
3,예스24티켓,4,뮤지컬 〈렛미플라이〉,2023.09.26~2023.12.10,예스24스테이지 1관,http://tkfile.yes24.com/upload2/perfblog/20231...,http://ticket.yes24.com/Perf/46671,2023-11-14
4,예스24티켓,5,뮤지컬 〈컴프롬어웨이〉,2023.11.28~2024.02.18,광림아트센터 BBCH홀,http://tkfile.yes24.com/upload2/perfblog/20230...,http://ticket.yes24.com/Perf/47295,2023-11-14
5,예스24티켓,6,2023 푸에르자부르타 웨이라 인 서울,2023.11.17~2024.02.15,성수문화예술마당 FB씨어터,http://tkfile.yes24.com/upload2/perfblog/20231...,http://ticket.yes24.com/Perf/47116,2023-11-14
6,예스24티켓,7,뮤지컬 〈비더슈탄트〉,2023.09.12~2023.11.26,대학로 드림아트센터 1관,http://tkfile.yes24.com/upload2/perfblog/20230...,http://ticket.yes24.com/Perf/46733,2023-11-14
7,예스24티켓,8,뮤지컬 〈키다리 아저씨〉,2023.12.05~2024.02.25,대학로 드림아트센터 1관,http://tkfile.yes24.com/upload2/perfblog/20231...,http://ticket.yes24.com/Perf/47438,2023-11-14
8,예스24티켓,9,어른이 뮤지컬 〈난쟁이들〉,2023.11.01~2024.01.21,플러스씨어터,http://tkfile.yes24.com/upload2/perfblog/20230...,http://ticket.yes24.com/Perf/47311,2023-11-14
9,예스24티켓,10,화가시리즈 - 뮤지컬 〈에곤 실레〉,2023.12.09~2024.03.10,서경대학교 공연예술센터 스콘2관,http://tkfile.yes24.com/upload2/perfblog/20231...,http://ticket.yes24.com/Perf/47421,2023-11-14


# `ticketlink_df, interpark_df, yes24_df` 을 이용해 `musical_rank Table` 삽입

In [151]:
host = 'ifive-db.ckteh9hwnkjf.ap-northeast-2.rds.amazonaws.com'

pwd = 'ifive1234'

db = 'ifive'

In [152]:
try:
    con = pymysql.connect(host=host, user='admin', password=pwd,  db=db, charset='utf8', port=3306)
    cur = con.cursor()
except Exception as e:
        print (e)(e)

In [153]:
cur.execute("show tables")

3

In [154]:
sql = "INSERT INTO musical_rank(site_name, ranking, title, date, location, poster_url, goods_url, update_date) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)"

In [163]:
# 인터파크 데이터 삽입
for idx, row in interpark_df.iterrows():
    try:
        print(tuple(row.values))
        cur.execute(sql,tuple(row.values))
    except Exception as e:
        print(e)

con.commit()

('인터파크', 1, '뮤지컬 드라큘라 (Dracula：The Musical)', '2023.12.6~2024.3.3', '샤롯데씨어터', 'https://tickets.interpark.com/contents/_next/image?url=https%3A%2F%2Fticketimage.interpark.com%2FPlay%2Fimage%2Flarge%2F23%2F23013512_p.gif&w=3840&q=75', 'https://tickets.interpark.com/goods/23013512', '2023-11-14')
('인터파크', 2, '태양의서커스 〈루치아〉', '2023.10.25~12.31', '잠실종합운동장 내 빅탑', 'https://tickets.interpark.com/contents/_next/image?url=https%3A%2F%2Fticketimage.interpark.com%2FPlay%2Fimage%2Flarge%2F23%2F23008707_p.gif&w=3840&q=75', 'https://tickets.interpark.com/goods/23008707', '2023-11-14')
('인터파크', 3, '뮤지컬 〈렌트〉', '2023.11.11~2024.2.25', 'coex 신한카드 artium', 'https://tickets.interpark.com/contents/_next/image?url=https%3A%2F%2Fticketimage.interpark.com%2FPlay%2Fimage%2Flarge%2F23%2F23013171_p.gif&w=3840&q=75', 'https://tickets.interpark.com/goods/23013171', '2023-11-14')
('인터파크', 4, '뮤지컬 〈몬테크리스토〉', '2023.11.21~ 2024.2.25', '충무아트센터 대극장', 'https://tickets.interpark.com/contents/_next/image?url=https%3A%2F%2Fti

In [164]:
# 티켓링크 데이터 삽입
for idx, row in ticketlink_df.iterrows():
    try:
        print(tuple(row.values))
        cur.execute(sql,tuple(row.values))
    except Exception as e:
        print(e)

con.commit()

('티켓링크', 1, '뮤지컬 드라큘라 (Dracula：The Musical)', '2023.12.05. ~ 2024.03.02.', '샤롯데씨어터', 'https://image.toast.com/aaaaab/ticketlink/TKL_5/23-DRC_POSTER_1020.jpg', 'https://www.ticketlink.co.kr/product/46101', '2023-11-14')
('티켓링크', 2, '뮤지컬 <문스토리>', '2023.09.25. ~ 2023.12.09.', '드림아트센터 4관', 'https://image.toast.com/aaaaab/ticketlink/TKL_3/main0821(1).jpg', 'https://www.ticketlink.co.kr/product/45428', '2023-11-14')
('티켓링크', 3, '뮤지컬 <타오르는 어둠 속에서>', '2023.08.25. ~ 2023.11.25.', '링크아트센터 페이코홀', 'https://image.toast.com/aaaaab/ticketlink/TKL_3/darkness_main_0825.jpg', 'https://www.ticketlink.co.kr/product/44995', '2023-11-14')
('티켓링크', 4, '뮤지컬 <시스터 액트>', '2023.11.20. ~ 2024.02.10.', '디큐브 링크아트센터', 'https://image.toast.com/aaaaab/ticketlink/TKL_1/m1019.jpg', 'https://www.ticketlink.co.kr/product/46006', '2023-11-14')
('티켓링크', 5, '뮤지컬 <스모크>', '2023.11.14. ~ 2024.02.03.', '링크아트센터 벅스홀', 'https://image.toast.com/aaaaab/ticketlink/TKL_9/main1003.jpg', 'https://www.ticketlink.co.kr/product/46233', '2023

In [165]:
# yes24티켓 데이터 삽입
for idx, row in yes24_df.iterrows():
    try:
        print(tuple(row.values))
        cur.execute(sql,tuple(row.values))
    except Exception as e:
        print(e)

con.commit()

('예스24티켓', '1', '뮤지컬 〈드라이 플라워〉', '2023.11.07~2024.01.07', '대학로 드림아트센터 3관', 'http://tkfile.yes24.com/upload2/perfblog/202311/20231113/20231113-47290.jpg', 'http://ticket.yes24.com/Perf/47290', '2023-11-14')
('예스24티켓', '2', '태양의서커스 〈루치아〉', '2023.10.25~2023.12.31', '잠실종합운동장 내 빅탑', 'http://tkfile.yes24.com/upload2/perfblog/202308/20230804/20230804-46305.jpg', 'http://ticket.yes24.com/Perf/46305', '2023-11-14')
('예스24티켓', '3', '오페레타 뮤지컬 〈판타지아〉 겨울시즌', '2023.11.11~2023.12.31', '국립중앙박물관 내 극장용', 'http://tkfile.yes24.com/upload2/perfblog/202311/20231113/20231113-47090.jpg', 'http://ticket.yes24.com/Perf/47090', '2023-11-14')
('예스24티켓', '4', '뮤지컬 〈렛미플라이〉', '2023.09.26~2023.12.10', '예스24스테이지 1관', 'http://tkfile.yes24.com/upload2/perfblog/202311/20231113/20231113-46671.jpg', 'http://ticket.yes24.com/Perf/46671', '2023-11-14')
('예스24티켓', '5', '뮤지컬 〈컴프롬어웨이〉', '2023.11.28~2024.02.18', '광림아트센터 BBCH홀', 'http://tkfile.yes24.com/upload2/perfblog/202309/20230925/20230925-47295.jpg', 'http://ticket.yes24.co